# Dynamic Recurrent Neural Network.

Implementación de TensorFlow de una Red Neural Recurrente (LSTM) que realiza computación dinámica sobre secuencias de longitud variable. Este ejemplo utiliza un conjunto de datos de juguetes para clasificar secuencias lineales. Las secuencias generadas tienen longitud variable.

## RNN

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png" alt="nn" style="width: 600px;"/>

Referencias:
- [Long Short Term Memory](http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf), Sepp Hochreiter & Jurgen Schmidhuber, Neural Computation 9(8): 1735-1780, 1997.

In [1]:
from __future__ import print_function

import tensorflow as tf
import random

In [2]:
# ====================
#  GENERADOR DE DATOS
# ====================

class ToySequenceData(object):
    """ Generar secuencia de datos con longitud dinámica.
    Esta clase genera muestras para el entrenamiento:
    - Clase 0: secuencias lineales (es decir,[0, 1, 2, 3,....])
    - Clase 1: secuencias aleatorias (es decir. [1, 3, 10, 7,...])

   AVISO:
    Tenemos que rellenar cada secuencia para alcanzar 'max_seq_len' para TensorFlow
    consistencia (no podemos alimentar una matriz numérica con inconsistentes
    dimensiones). El cálculo dinámico se llevará a cabo gracias a
    Seqlen' atributo que registra cada longitud de secuencia real.
    """
    def __init__(self, n_samples=1000, max_seq_len=20, min_seq_len=3,
                 max_value=1000):
        self.data = []
        self.labels = []
        self.seqlen = []
        for i in range(n_samples):
            # Longitud de secuencia aleatoria
            len = random.randint(min_seq_len, max_seq_len)
            # Monitorización de la longitud de la secuencia para el cálculo dinámico de TensorFlow
            self.seqlen.append(len)
            # Añadir una secuencia int aleatoria o lineal (50% prob)
            if random.random() < .5:
                # Generar una secuencia lineal
                rand_start = random.randint(0, max_value - len)
                s = [[float(i)/max_value] for i in
                     range(rand_start, rand_start + len)]
                # Secuencia de almohadillas para una consistencia de dimensiones
                s += [[0.] for i in range(max_seq_len - len)]
                self.data.append(s)
                self.labels.append([1., 0.])
            else:
                # Generar una secuencia aleatoria
                s = [[float(random.randint(0, max_value))/max_value]
                     for i in range(len)]
                # Secuencia de almohadillas para una consistencia de dimensiones
                s += [[0.] for i in range(max_seq_len - len)]
                self.data.append(s)
                self.labels.append([0., 1.])
        self.batch_id = 0

    def next(self, batch_size):
        """ Return a batch of data. When dataset end is reached, start over.
        """
        if self.batch_id == len(self.data):
            self.batch_id = 0
        batch_data = (self.data[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        batch_labels = (self.labels[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        batch_seqlen = (self.seqlen[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        self.batch_id = min(self.batch_id + batch_size, len(self.data))
        return batch_data, batch_labels, batch_seqlen

In [3]:
# ==========
#   MODELO
# ==========

# Parametros
learning_rate = 0.01
training_steps = 10000
batch_size = 128
display_step = 200

# Network Parametros
seq_max_len = 20 # Secuencia de longitud máxima
n_hidden = 64 # capa oculta número de características
n_classes = 2 # secuencia lineal o no

trainset = ToySequenceData(n_samples=1000, max_seq_len=seq_max_len)
testset = ToySequenceData(n_samples=500, max_seq_len=seq_max_len)

# tf Entrada de gráficos
x = tf.placeholder("float", [None, seq_max_len, 1])
y = tf.placeholder("float", [None, n_classes])
# Un placeholder para indicar la longitud de cada secuencia
seqlen = tf.placeholder(tf.int32, [None])

# Definir pesos
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [4]:
def dynamicRNN(x, seqlen, weights, biases):

    # Preparar la forma de los datos para que coincida con los requisitos de la función `rnn
    # Forma actual de entrada de datos: (tamaño_de_lote, n_pasos, n_input)
    # Forma requerida: ` n_pasos' tensores lista de formas (batch_size, n_input)
    
    # Desapilado para obtener una lista de tensores de forma'n_pasos' (batch_size, n_input)
    x = tf.unstack(x, seq_max_len, 1)

    # Define una célula lstm con tensorflow
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)

    # Obtenga la salida de la celda lstm, siempre y cuando 'sequence_length' se ejecute de forma dinámica
    # cálculo.
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32,
                                sequence_length=seqlen)

    # Cuando realizamos cálculos dinámicos, debemos recuperar el último
    # salida calculada dinámicamente, es decir, si la longitud de una secuencia es 10, necesitamos
    # para recuperar la décima salida.
    # Sin embargo, TensorFlow aún no soporta indexación avanzada, por lo que creamos
    # una opción personalizada que para cada muestra en el tamaño de lote, obtener su longitud y
    # obtener la salida correspondiente.

    #'outputs' es una lista de salidas en cada paso del tiempo, las empaquetamos en un Tensor
    # y volver a cambiar la dimensión a[batch_size, n_step, n_input]
    outputs = tf.stack(outputs)
    outputs = tf.transpose(outputs, [1, 0, 2])

    # Hack para construir la indexación y recuperar la salida correcta.
    batch_size = tf.shape(outputs)[0]
    # Índices de inicio para cada muestra
    index = tf.range(0, batch_size) * seq_max_len + (seqlen - 1)
    # Indexear
    outputs = tf.gather(tf.reshape(outputs, [-1, n_hidden]), index)

    # Activación lineal, utilizando las salidas calculadas anteriormente
    return tf.matmul(outputs, weights['out']) + biases['out']

In [5]:
pred = dynamicRNN(x, seqlen, weights, biases)

# Definir la pérdida y el optimizador
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluaar modelo
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Inicializar las variables (es decir, asignar su valor por defecto)
init = tf.global_variables_initializer()

/Users/aymeric.damien/anaconda2/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [6]:
# Iniciar entrenamiento
with tf.Session() as sess:

    # Ejecutar el inicializador
    sess.run(init)

    for step in range(1, training_steps+1):
        batch_x, batch_y, batch_seqlen = trainset.next(batch_size)
        # Optimización de la ejecución (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       seqlen: batch_seqlen})
        if step % display_step == 0 or step == 1:
            # Calcular la precisión y la pérdida del lote
            acc, loss = sess.run([accuracy, cost], feed_dict={x: batch_x, y: batch_y,
                                                seqlen: batch_seqlen})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))

    print("Optimization Finished!")

    # Calcular la precisión
    test_data = testset.data
    test_label = testset.labels
    test_seqlen = testset.seqlen
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label,
                                      seqlen: test_seqlen}))

Step 1, Minibatch Loss= 0.864517, Training Accuracy= 0.42188
Step 200, Minibatch Loss= 0.686012, Training Accuracy= 0.43269
Step 400, Minibatch Loss= 0.682970, Training Accuracy= 0.48077
Step 600, Minibatch Loss= 0.679640, Training Accuracy= 0.50962
Step 800, Minibatch Loss= 0.675208, Training Accuracy= 0.53846
Step 1000, Minibatch Loss= 0.668636, Training Accuracy= 0.56731
Step 1200, Minibatch Loss= 0.657525, Training Accuracy= 0.62500
Step 1400, Minibatch Loss= 0.635423, Training Accuracy= 0.67308
Step 1600, Minibatch Loss= 0.580433, Training Accuracy= 0.75962
Step 1800, Minibatch Loss= 0.475599, Training Accuracy= 0.81731
Step 2000, Minibatch Loss= 0.434865, Training Accuracy= 0.83654
Step 2200, Minibatch Loss= 0.423690, Training Accuracy= 0.85577
Step 2400, Minibatch Loss= 0.417472, Training Accuracy= 0.85577
Step 2600, Minibatch Loss= 0.412906, Training Accuracy= 0.85577
Step 2800, Minibatch Loss= 0.409193, Training Accuracy= 0.85577
Step 3000, Minibatch Loss= 0.406035, Training A